In [1]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers
!pip install datasets
!pip install evaluate
%pip install peft==0.4.0

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from datasets import Dataset, load_metric
import evaluate
import matplotlib.pyplot as plt
import torch
import os
import peft
from peft import LoraConfig, get_peft_model,TaskType

In [3]:
num_cpus = os.cpu_count()
print(f"Số lượng CPU có trong máy: {num_cpus}")

Số lượng CPU có trong máy: 4


In [4]:
fields_path = "/kaggle/input/fieldquestionvnlaw/fields_datalaw.json"

In [5]:
fields_dataset = pd.read_json(fields_path)
fields_dataset.head(2)

,question,Bảo_hiểm,Bất_động_sản,Bộ_máy hành_chính,Chứng_khoán,Công_nghệ_thông_tin,Doanh_nghiệp,Dịch_vụ pháp_lý,Giao_thông - Vận_tải,Giáo_dục,...,Thể_thao - Y_tế,Thủ_tục Tố_tụng,Tiền_tệ - Ngân_hàng,Trách_nhiệm hình_sự,Tài_chính nhà_nước,Vi_phạm hành_chính,Văn_hoá - Xã_hội,Xuất_nhập_khẩu,Xây_dựng - Đô_thị,Đầu_tư
0,Khi doanh_nghiệp trong nước hợp_tác với nhà_đầ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,Nhà_đầu_tư nước_ngoài được áp_dụng điều_kiện t...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
len(fields_dataset)

147660

In [ ]:
# Split the dataset into train and remaining (80% - 20%)
train_fields_dataset = fields_dataset.sample(frac=0.8, random_state=42)
remaining_dataset = fields_dataset.drop(train_fields_dataset.index)

# Split the remaining dataset into validation and test (50% - 50%)
validation_fields_dataset = remaining_dataset.sample(frac=0.5, random_state=42)
test_fields_dataset = remaining_dataset.drop(validation_fields_dataset.index)

# Print the number of records in each set
print(f'The training fields dataset has {len(train_fields_dataset)} records.')
print(f'The validation fields dataset has {len(validation_fields_dataset)} records.')
print(f'The test fields dataset has {len(test_fields_dataset)} records.')

In [ ]:
labels = [label for label in train_fields_dataset.columns if label != 'question']
id2label = {idx: label for idx, label in enumerate(labels)}
label2id = {label: idx for idx, label in enumerate(labels)}
#labels

In [ ]:
question_length = fields_dataset['question'].apply(len)
average_length = question_length.mean()
min_length = question_length.min()
max_length = question_length.max()

print(f"Trung bình độ dài: {average_length}")
print(f"Độ dài ngắn nhất: {min_length}")
print(f"Độ dài dài nhất: {max_length}")

In [ ]:
plt.hist(question_length, bins=10, edgecolor='k')
plt.xlabel('Độ dài câu')
plt.ylabel('Số lượng câu')
plt.title('Phân phối độ dài câu trong "question"')
plt.show()

In [ ]:
hg_train_fields_dataset = Dataset.from_pandas(train_fields_dataset)
hg_test_fields_dataset = Dataset.from_pandas(test_fields_dataset)
hg_validation_fields_dataset = Dataset.from_pandas(validation_fields_dataset)

In [ ]:
print(f'The length of hg_train_fields_dataset is {len(hg_train_fields_dataset)}.\n')
hg_train_fields_dataset[0]

In [ ]:
fields_dataset.iloc[[85682]]

In [ ]:
model_name = "vinai/phobert-base-v2"
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
print(f'The unknown token is {tokenizer.unk_token} and the ID for the unknown token is {tokenizer.unk_token_id}')
print(f'The seperator token is {tokenizer.sep_token} and the ID for the seperator token is {tokenizer.sep_token_id}')
print(f'The pad token is {tokenizer.pad_token} and the ID for the pad token is {tokenizer.pad_token_id}')
print(f'The sentence level classification token is {tokenizer.cls_token} and the ID for the classification token is {tokenizer.cls_token_id}')
print(f'The mask token is {tokenizer.mask_token} and the ID for the mask token is {tokenizer.mask_token_id}')

In [ ]:
def tokenize_fields_dataset(examples):
  # take a batch of texts
  text = examples["question"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=256)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [ ]:
encoded_dataset_fields_train = hg_train_fields_dataset.map(tokenize_fields_dataset, batched=True, remove_columns=hg_train_fields_dataset.column_names)
encoded_dataset_fields_validation = hg_validation_fields_dataset.map(tokenize_fields_dataset, batched=True, remove_columns=hg_train_fields_dataset.column_names)

In [ ]:
encoded_dataset_fields_test = hg_test_fields_dataset.map(tokenize_fields_dataset, batched=True, remove_columns=hg_train_fields_dataset.column_names)

In [ ]:
encoded_dataset_fields_train.set_format("torch")
encoded_dataset_fields_validation.set_format("torch")
encoded_dataset_fields_test.set_format("torch")

In [ ]:
example = encoded_dataset_fields_train[0]
print(example.keys())

In [ ]:
tokenizer.decode(example['input_ids'])

In [ ]:
example['labels']

In [ ]:
example['input_ids']

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
foundation_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(labels), id2label=id2label, label2id=label2id, torch_dtype=torch.bfloat16).to(device)

In [ ]:
metric_name = "f1"

In [ ]:
training_args = TrainingArguments(
    output_dir="./fields_classification",
    logging_dir="./fields_classification/logs",
    evaluation_strategy='epoch',
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=4,
    eval_accumulation_steps = 2,
    eval_delay=0.5,
    learning_rate=3e-5,
    weight_decay=1e-4,
    max_grad_norm=1.0,
    num_train_epochs=10,
    lr_scheduler_type="reduce_lr_on_plateau", # Giảm khi hiệu suất không cải thiện trên tập validation.
    warmup_ratio=0.1,
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    log_level='debug',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=5,
    save_safetensors=True,
    seed=42,
    fp16=False,
    dataloader_num_workers=num_cpus,
    run_name="PhoBERTv4",
    load_best_model_at_end=True,
    metric_for_best_model=metric_name
)

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, precision_score, recall_score
from transformers import EvalPrediction
import torch

def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply Sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    precision_micro_average = precision_score(y_true=y_true, y_pred=y_pred, average='micro')
    recall_micro_average = recall_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {
        'f1': f1_micro_average,
        'precision': precision_micro_average,
        'recall': recall_micro_average,
        'roc_auc': roc_auc,
        'accuracy': accuracy
    }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

In [ ]:
encoded_dataset_fields_train[0]['labels'].type()

In [ ]:
encoded_dataset_fields_train[0]['input_ids']

In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=encoded_dataset_fields_train,
    eval_dataset=encoded_dataset_fields_validation,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)])

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
qa = hg_test_fields_dataset[200]
print(qa)
example = encoded_dataset_fields_test[200]
true_labels = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
print(true_labels)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/checkpoint-9020/checkpoint-9020")

In [ ]:
training_args = TrainingArguments(
    output_dir="./fields_classification",
    logging_dir="./fields_classification/logs",
    logging_strategy='epoch',
    logging_steps=100,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-6,
    weight_decay=0.1,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset_fields_train,
    eval_dataset=encoded_dataset_fields_validation,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)])

In [ ]:
print(trainer.state.best_model_checkpoint)

In [ ]:
print(len(hg_test_fields_dataset))
num_test = 11
question = hg_test_fields_dataset["question"][num_test]
example = encoded_dataset_fields_test[num_test]
encoding = tokenizer(question, return_tensors="pt")
encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

outputs = trainer.model(**encoding)

logits = outputs.logits
logits.shape

sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu()).detach().numpy()  # Detach the tensor and convert to a NumPy array
print(probs)
predictions = np.zeros(probs.shape)
predictions[np.argmax(probs)] = 1

# Turn predicted id's into actual label names
predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
print(predicted_labels)
true_labels = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]
print(true_labels)

In [ ]:
from tqdm import tqdm

correct_count = 0
total_count = len(hg_test_fields_dataset)

for num_test in tqdm(range(total_count), desc="Processing examples"):
    question = hg_test_fields_dataset[num_test]['question']
    example = encoded_dataset_fields_test[num_test]
    encoding = tokenizer(question, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k, v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu()).detach().numpy()
    
    predictions = np.zeros(probs.shape)
    predictions[np.argmax(probs)] = 1

    # Turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
    
    true_labels = [id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

    if set(predicted_labels) == set(true_labels):
        correct_count += 1

accuracy = correct_count / total_count
print(f"Correct predictions: {correct_count} out of {total_count}")
print(f"Accuracy: {accuracy:.2%}")

In [ ]:
id2label

In [ ]:
!pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.model.push_to_hub("legal-vn-field-v2")